[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/combine-org/combine-notebooks/main?labpath=notebooks%2Fsbgn.ipynb)
<a href="https://colab.research.google.com/github/combine-org/combine-notebooks/blob/main/notebooks/sbgn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab\"/></a>

# Simple SBGN example

This notebook creates a simple model in [SBGN](https://sbgn.github.io/specifications). It models an irreversible reacton. The enzyme catalyses an irreversible metabolic process which consumes substrate S1 and produces product P1. The enzyme is a represented as a macromolecule connected to the process glyph by a catalysis arc. The substrate and the product of the biochemical reaction are represented by simple chemical glyphs. Then we renders a graphical representation and print the SBGN xml.

## 1) Including libraries

Note: Please uncomment the line below if you use the Google Colab.

In [ ]:
#%pip install git+https://github.com/combine-org/combine-notebooks

In [1]:
from pathlib import Path

from IPython.display import Image
import libsbgnpy.libsbgn as libsbgn
from libsbgnpy.libsbgnTypes import ArcClass, GlyphClass, Language
from libsbgnpy import render
import copy

from combine_notebooks import RESULTS_DIR

In [2]:
def get_sbgn_string(doc_before : libsbgn.sbgn):
    # Creates temporary sbgn file to extract xml as a string
    RESULTS_DIR.mkdir(parents=True, exist_ok=True)
    sbgn_path = RESULTS_DIR / "temp.sbgn"
    f_out = sbgn_path
    doc_before.write_file(f_out)
    xml = open(f_out).read()
    Path.unlink(sbgn_path) 
    return xml

def pretty_print(sbgn_before : libsbgn.sbgn, sbgn_after : libsbgn.sbgn = None):
    # Break the original SBML into lines
    original_sbgn = get_sbgn_string(sbgn_before)
    original_sbgn_lines = original_sbgn.split('\n')
    # Print the entire document in red
    if sbgn_after == None:
        original_document = '\n'.join(original_sbgn_lines)
        print("\x1b[31m" + original_document + "\x1b[0m") # ANSI escape used to print colours here
    # If a new document is also given, highlight the lines that have changed
    else:
        # Split the updated SBML into lines
        new_sbgn = get_sbgn_string(sbgn_after)
        new_sbgn_lines = new_sbgn.split('\n')
        # Iterate over each new line
        for new_line in new_sbgn_lines:
            # Print any new lines in red, otherwise print normally
            if new_line in original_sbgn_lines:
                print(new_line)
            else:
                print("\x1b[31m" + new_line + "\x1b[0m") # ANSI escape used to print colours here

## 2) Declaring the SBGN model

Create empty sbgn.

In [3]:
sbgn: libsbgn.sbgn = libsbgn.sbgn()

pretty_print(sbgn)

<?xml version="1.0" encoding="UTF-8"?>
<sbgn/>



Create map, set language and set in sbgn.

In [4]:
start_sbgn = copy.deepcopy(sbgn)

map = libsbgn.map()
map.set_language(Language.PD)
sbgn.set_map(map)

pretty_print(start_sbgn, sbgn)

<?xml version="1.0" encoding="UTF-8"?>
<sbgn xmlns="http://sbgn.org/libsbgn/0.2">
    <map language="process description"/>
</sbgn>



Create four glyphs.  We label three of the glyphs as S1, P1, and enzyme.

In [5]:
start_sbgn = copy.deepcopy(sbgn)

g = libsbgn.glyph(class_=GlyphClass.SIMPLE_CHEMICAL, id="glyph0")
g.set_label(libsbgn.label(text="S1"))
g.set_bbox(libsbgn.bbox(y="105.0", x="25.0", h="60.0", w="60.0"))
map.add_glyph(g)

g = libsbgn.glyph(class_=GlyphClass.MACROMOLECULE, id="glyph2")
g.set_label(libsbgn.label(text="enzyme"))
g.set_bbox(libsbgn.bbox(y="30.0", x="115.0", h="40.0", w="80.0"))
map.add_glyph(g)

g = libsbgn.glyph(class_=GlyphClass.PROCESS, id="glyph3")
g.set_bbox(libsbgn.bbox(y="123.0", x="143.0", h="24.0", w="24.0"))
g.add_port(libsbgn.port(y="135.0", x="131.0", id="glyph3.1"))
g.add_port(libsbgn.port(y="135.0", x="179.0", id="glyph3.2"))
map.add_glyph(g)

g = libsbgn.glyph(class_=GlyphClass.SIMPLE_CHEMICAL, id="glyph1")
g.set_label(libsbgn.label(text="P1"))
g.set_bbox(libsbgn.bbox(y="105.0", x="225.0", h="60.0", w="60.0"))
map.add_glyph(g)

pretty_print(start_sbgn, sbgn)

<?xml version="1.0" encoding="UTF-8"?>
<sbgn xmlns="http://sbgn.org/libsbgn/0.2">
    <map language="process description">
        <glyph class="simple chemical" id="glyph0">
            <label text="S1"/>
            <bbox w="60." h="60." x="25." y="105."/>
        </glyph>
        <glyph class="macromolecule" id="glyph2">
            <label text="enzyme"/>
            <bbox w="80." h="40." x="115." y="30."/>
        </glyph>
        <glyph class="process" id="glyph3">
            <bbox w="24." h="24." x="143." y="123."/>
            <port id="glyph3.1" x="131." y="135."/>
            <port id="glyph3.2" x="179." y="135."/>
        </glyph>
        <glyph class="simple chemical" id="glyph1">
            <label text="P1"/>
            <bbox w="60." h="60." x="225." y="105."/>
        </glyph>
    </map>
</sbgn>



Add three arcs linking the glyphs together.

In [6]:
start_sbgn = copy.deepcopy(sbgn)

a = libsbgn.arc(class_=ArcClass.CONSUMPTION, target="glyph3.1", source="glyph0", id="arc0")
a.set_start(libsbgn.startType(y="135.0", x="85.0"))
a.set_end(libsbgn.endType(y="135.0", x="131.0"))
map.add_arc(a)

a = libsbgn.arc(class_=ArcClass.CATALYSIS, target="glyph3", source="glyph2", id="arc1")
a.set_start(libsbgn.startType(y="70.0", x="155.0"))
a.set_end(libsbgn.endType(y="123.0", x="155.0"))
map.add_arc(a)

a = libsbgn.arc(class_=ArcClass.PRODUCTION, target="glyph1", source="glyph3.2", id="arc2")
a.set_start(libsbgn.startType(y="135.0", x="179.0"))
a.set_end(libsbgn.endType(y="135.0", x="225.0"))
map.add_arc(a)

pretty_print(start_sbgn, sbgn)

<?xml version="1.0" encoding="UTF-8"?>
<sbgn xmlns="http://sbgn.org/libsbgn/0.2">
    <map language="process description">
        <glyph class="simple chemical" id="glyph0">
            <label text="S1"/>
            <bbox w="60." h="60." x="25." y="105."/>
        </glyph>
        <glyph class="macromolecule" id="glyph2">
            <label text="enzyme"/>
            <bbox w="80." h="40." x="115." y="30."/>
        </glyph>
        <glyph class="process" id="glyph3">
            <bbox w="24." h="24." x="143." y="123."/>
            <port id="glyph3.1" x="131." y="135."/>
            <port id="glyph3.2" x="179." y="135."/>
        </glyph>
        <glyph class="simple chemical" id="glyph1">
            <label text="P1"/>
            <bbox w="60." h="60." x="225." y="105."/>
        </glyph>
        <arc class="consumption" id="arc0" source="glyph0" target="glyph3.1">
            <start x="85." y="135."/>
            <end x="131." y="135."/>
        </arc>
        <arc class="catalysi

## 3) Write, print and render the generated file 

Save SBGN model as text and then print that model.

In [7]:
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
sbgn_path = RESULTS_DIR / "hello_world_sbgn.sbgn"
f_out = sbgn_path
sbgn.write_file(f_out)
xml = open(f_out).read()
print(xml)

<?xml version="1.0" encoding="UTF-8"?>
<sbgn xmlns="http://sbgn.org/libsbgn/0.2">
    <map language="process description">
        <glyph class="simple chemical" id="glyph0">
            <label text="S1"/>
            <bbox w="60." h="60." x="25." y="105."/>
        </glyph>
        <glyph class="macromolecule" id="glyph2">
            <label text="enzyme"/>
            <bbox w="80." h="40." x="115." y="30."/>
        </glyph>
        <glyph class="process" id="glyph3">
            <bbox w="24." h="24." x="143." y="123."/>
            <port id="glyph3.1" x="131." y="135."/>
            <port id="glyph3.2" x="179." y="135."/>
        </glyph>
        <glyph class="simple chemical" id="glyph1">
            <label text="P1"/>
            <bbox w="60." h="60." x="225." y="105."/>
        </glyph>
        <arc class="consumption" id="arc0" source="glyph0" target="glyph3.1">
            <start x="85." y="135."/>
            <end x="131." y="135."/>
        </arc>
        <arc class="catalysi

Render the SBGN model.

In [ ]:
# render SBGN
f_png = RESULTS_DIR / 'hello_world_sbgn.png'
render.render_sbgn(sbgn, image_file=str(f_png), file_format="png")
Image(f_png, width=300)